In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
import tkinter
from tkinter import *
import time
import PIL.Image, PIL.ImageTk

In [4]:
unique_labels = [
    'agriculture', 
    'artisinal_mine', 
    'bare_ground', 
    'blooming', 
    'blow_down', 
    'clear', 
    'cloudy', 
    'conventional_mine', 
    'cultivation', 
    'habitation', 
    'haze', 
    'partly_cloudy', 
    'primary', 
    'road', 
    'selective_logging', 
    'slash_burn', 
    'water'
]

In [5]:
def oneHotEncode(labels_list):
    onehot = [];
    for label in unique_labels: 
        onehot.append(int(label in labels_list))
    return onehot    

def oneHotDecode(labels_list):
    labels = []
    for i, probability in enumerate(labels_list):
        if probability.item() > 0.7:
            labels.append(unique_labels[i])
    return labels

In [6]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=2, stride=1, padding='same')
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=2, stride=1, padding='same')
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=2, stride=1, padding='same')
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=2, stride=1, padding='same')
        self.fc1 = nn.Linear(256 * 8 * 8, 1000)
        self.fc2 = nn.Linear(1000, 500)
        self.fc3 = nn.Linear(500, len(unique_labels))
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Model()
model.to(device)

criterion = nn.BCEWithLogitsLoss (weight=None, reduce=True)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

/home/cristine/.local/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [7]:
model = Model()
model.load_state_dict(torch.load('model/tensor.pth'))
model.to(device)
model.eval()

Model(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1): Conv2d(3, 32, kernel_size=(2, 2), stride=(1, 1), padding=same)
  (conv2): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1), padding=same)
  (conv3): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1), padding=same)
  (conv4): Conv2d(128, 256, kernel_size=(2, 2), stride=(1, 1), padding=same)
  (fc1): Linear(in_features=16384, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=17, bias=True)
)

In [8]:
def predict(image):
    
    transform = transforms.Compose(
        [
            transforms.Resize((128, 128)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(degrees=45),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

    # Convert to tensor
    inputs = transform(image).to('cuda').unsqueeze(0)
    
    #predict 
    outputs = model(inputs)
    outputs = torch.sigmoid(outputs)
    return oneHotDecode(outputs[0])


In [ ]:
#prediction = predict('/home/cristine/Imagens/mato_grosso_tes.jpg')
#print(prediction)

In [9]:
def returnLeftRight(x, width):
    x = 250 if x < 250 else ((width - 250) if (x + 250 > width) else x)
    left = x - 250
    right = x + 250
    return left, right

def returnTopBottom(y, height):
    y = 250 if y < 250 else ((height - 250) if (y + 250 > height) else y)
    top = y - 250
    bottom = y + 250
    return top, bottom

def crop_image(x, y):
    im = PIL.Image.open("/home/cristine/Imagens/mato_grosso.jpg")
    
    width, height = im.size
    left, right = returnLeftRight(x, width)
    top, bottom = returnTopBottom(y, height) 
    im1 = im.crop((left, top, right, bottom))
    return im1



In [ ]:
# Create a window
window = tkinter.Tk()
window.geometry('1000x600')
window.maxsize(1000, 600)

frame = tkinter.Frame(window, relief='raised') # relief=SUNKEN)
frame.grid_rowconfigure(0, weight=1)
frame.grid_columnconfigure(0, weight=1)

xscrollbar = tkinter.Scrollbar(frame, orient=tkinter.HORIZONTAL)
xscrollbar.grid(row=1, column=0, sticky=tkinter.E+tkinter.W)

yscrollbar = tkinter.Scrollbar(frame)
yscrollbar.grid(row=0, column=1, sticky=tkinter.N+tkinter.S)

canvas = tkinter.Canvas(frame, bd=0, xscrollcommand=xscrollbar.set, yscrollcommand=yscrollbar.set, 
                        width=1000, height=600)
canvas.grid(row=0, column=0, sticky=tkinter.N+tkinter.S+tkinter.E+tkinter.W)

File = "/home/cristine/Imagens/mato_grosso.jpg"
img = PIL.ImageTk.PhotoImage(PIL.Image.open(File))

canvas.create_image(0,0,image=img, anchor="nw")
canvas.config(scrollregion=canvas.bbox(tkinter.ALL))

xscrollbar.config(command=canvas.xview)
yscrollbar.config(command=canvas.yview)

def on_button_pressed(event):
    canvas = event.widget
    x = canvas.canvasx(event.x)
    y = canvas.canvasy(event.y)
    cropped_image = crop_image(x, y)
    prediction = predict(cropped_image)
    label.config(text=", ".join(prediction))

canvas.bind("<Button-1>", on_button_pressed)


label = Label(
    window,
    text="",
    font=(21),
    padx=10,
    pady=5,
    bg='#d9d8d7'
    )

label.pack(expand=True)
frame.pack()
window.update()


while True:
    window.update()